Functional API:

zie boek H10 > implementing MLPs with keras > build complex models using the functional API

In [63]:
from functools import partial

import keras

In [64]:
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="same", kernel_initializer="he_normal",
                        use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()
            ]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

## 2.1 Implementing a Residual Unit

### 2.1.1 Rewrite ResidualUnit as a Method using the Functional API

In [65]:
def residual_unit(input_, filters, strides=1, activation="relu"):
  activation_layer = keras.activations.get(activation)

  """
  Lagen aanmaken is dit onderdeel in de call functie hierboven:

  Z = inputs
  for layer in self.main_layers:
      Z = layer(Z)
  """

  # Laag aanmaken en meteen aanroepen defcon(...)(...) dubbele haakjes = meteen aanroepen
  main = DefaultConv2D(filters, strides=strides)(input_)
  main = keras.layers.BatchNormalization()(main)
  main = activation_layer(main)
  main = DefaultConv2D(filters)(main)
  main = keras.layers.BatchNormalization()(main)

  """
  Skip laag aanmaken is dit onderdeel in de call functie hierboven:

  skip_Z = inputs
  for layer in self.skip_layers:
      skip_Z = layer(skip_Z)
  """
  skip = input_
  if strides > 1:
    skip = DefaultConv2D(filters, kernel_size=1, strides=strides)(skip)
    skip = keras.layers.BatchNormalization()(skip)

  """
  Activatielaag aanmaken is dit onderdeel in de call functie hierboven:

  return self.activation(Z + skip_Z)
  """

  return activation_layer(main + skip)

Testen of functional api code hetzelfde doet als de class

In [66]:
# Constante test shape: 128 hoog, 128 breed, 50 kanalen
TEST_SHAPE = (128 ,128 ,50)

# Eerste model is een eenvoudig sequentieel model die de class gebruikt
model1 = keras.Sequential([
  keras.layers.Input(shape=TEST_SHAPE),
  ResidualUnit(filters =50)
])

# Tweede model gebruikt functional API
input_ = keras.layers.Input(shape=TEST_SHAPE)
output = residual_unit(input_, filters=50)
model2 = keras.Model(inputs=input_, outputs=output)

In [67]:
model1.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ residual_unit_3 (ResidualUnit)  │ (None, 128, 128, 50)   │        45,400 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,400 (177.34 KB)

 Trainable params: 45,200 (176.56 KB)

 Non-trainable params: 200 (800.00 B)

2OO niet-trainbare parameters
* Komen van batch normalisatie laag
* Per laag: 50 niet trainbare parameters (mu-hoedje) + 50 worden sigma-hoedje

In [68]:
model2.summary()

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10      │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 128, 128,  │     22,500 │ input_layer_10[0… │
│                     │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        200 │ conv2d_20[0][0]   │
│ (BatchNormalizatio… │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_12 (ReLU)     │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_21 (Conv2D)  │ (None, 128, 128,  │     22,500 │ re_lu_12[0][0]    │
│                     │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        200 │ conv2d_21[0][0]   │
│ (BatchNormalizatio… │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 50)               │            │ input_layer_10[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_13 (ReLU)     │ (None, 128, 128,  │          0 │ add_6[0][0]       │
│                     │ 50)               │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 45,400 (177.34 KB)

 Trainable params: 45,200 (176.56 KB)

 Non-trainable params: 200 (800.00 B)

Parameters zijn alleszins al gelijk!

In [69]:
# Create a random tensor to serve as input.
# The tensor should have a batch size equal to one.
X = keras.random.normal(shape=(1, *TEST_SHAPE))
X.shape

TensorShape([1, 128, 128, 50])

In [70]:
# Check the shape of the output when calling model1 on X.
model1(X).shape

TensorShape([1, 128, 128, 50])

In [71]:
# Check the shape of the output when calling model2 on X.
model2(X).shape

TensorShape([1, 128, 128, 50])

In [72]:
# Shape van alle gewichten van model2 printen

for w in model2.get_weights():
  print(w.shape)

(3, 3, 50, 50)
(50,)
(50,)
(50,)
(50,)
(3, 3, 50, 50)
(50,)
(50,)
(50,)
(50,)


In [73]:
# Copy the weights from the second model into the first model.
model1.set_weights(model2.get_weights())

# !!!! Werkt enkel als lagen in zelfde volgorde staan !!!

In [74]:
# Combine keras.ops.isclose and keras.ops.all to check that the output of both models is now identical.
keras.ops.isclose(model1(X), model2(X))

<tf.Tensor: shape=(1, 128, 128, 50), dtype=bool, numpy=
array([[[[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True]],

        [[ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         [ True,  True,  True, ...,  True,  True,  True],
         ...,
         [ True,  True,  Tru

Gaat plaats per plaats gaan kijken of ze gelijk zijn

In [75]:
keras.ops.all(keras.ops.isclose(model1(X), model2(X)))

<tf.Tensor: shape=(), dtype=bool, numpy=True>

### 2.1.2 Write a method that returns a Model.

In [76]:
def residual_unit_model (input_shape, filters, strides =1, activation="relu", name=None) -> keras.Model:
  activation_layer = keras.activations.get(activation)

  input_ = keras.layers.Input(shape=input_shape)

  main = DefaultConv2D(filters, strides=strides)(input_)
  main = keras.layers.BatchNormalization()(main)
  main = activation_layer(main)
  main = DefaultConv2D(filters)(main)
  main = keras.layers.BatchNormalization()(main)

  skip = input_
  if strides > 1:
    skip = DefaultConv2D(filters, kernel_size=1, strides=strides)(skip)
    skip = keras.layers.BatchNormalization()(skip)

  output = activation_layer(main + skip)

  return keras.Model(inputs=input_, outputs=output)

In [77]:
model3 = residual_unit_model (input_shape=TEST_SHAPE, filters=50)

In [78]:
model3.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11      │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_22 (Conv2D)  │ (None, 128, 128,  │     22,500 │ input_layer_11[0… │
│                     │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        200 │ conv2d_22[0][0]   │
│ (BatchNormalizatio… │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_14 (ReLU)     │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_23 (Conv2D)  │ (None, 128, 128,  │     22,500 │ re_lu_14[0][0]    │
│                     │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        200 │ conv2d_23[0][0]   │
│ (BatchNormalizatio… │ 50)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 128, 128,  │          0 │ batch_normalizat… │
│                     │ 50)               │            │ input_layer_11[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_15 (ReLU)     │ (None, 128, 128,  │          0 │ add_7[0][0]       │
│                     │ 50)               │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 45,400 (177.34 KB)

 Trainable params: 45,200 (176.56 KB)

 Non-trainable params: 200 (800.00 B)

## 2.2 Build a Convolutional Layer from Scratch

Hier komt altijd een examenvraag uit!

In [79]:
class MyConv2D(keras.layers.Layer):
  # init = constructor
  def __init__ (self, filters: int, kernel_size: int, activation: str, **kwargs):
    super().__init__(** kwargs)
    self.filters = filters
    self.kernel_size = kernel_size
    self.activation = keras.activations.get(activation)

  # nodig voor flexibiliteit
  # om kernel aan te maken, moeten we weten wat het aantal kanalen is van de vorige laag
  # build wordt aangeroepen wanneer eerste keer call aangeroepen wordt
  # als die laag nog niet aangemaakt is, wordt build aangeroepen
  def build(self, batch_input_shape):
    print(f"Calling build", flush=True)
    # batch, height, width, channels_in = batch_input_shape[1:]
    # batch, height en width wordt niet echt gebruikt dus daarom vervangen door underscores
    _, _, channels_in = batch_input_shape[1:]

    """
    Uit de docs: https://keras.io/api/ops/nn/#conv-function
    -----------
    kernel:
      Tensor of rank N+2. kernel has shape (kernel_spatial_shape, num_input_channels, num_output_channels).
      num_input_channels should match the number of channels in inputs.
    -----------

    kernel_spatial_shape = (self.kernel_size, self.kernel_size) kernel_size is een int maar we hebben een shape nodig (hoogte en breedte)
    num_input_channels = channels_in van de batch_input_shape
    num_output_channels = aantal filters
    """

    self.kernel = self.add_weight(
        shape=(self.kernel_size, self.kernel_size, channels_in, self.filters),
        initializer="he_normal"
    )
    self.bias = self.add_weight(shape=(self.filters,), initializer="zeros")

    """
    Heel belangrijk op examen: hoe voeg je de leerbare gewichten toe met de juiste shape?
      Hoe je self.kernel en self.bias hier dus maakt
    """


  # call = wordt uitgevoerd wanneer je de laag wil toepassen op een tensor
  def call(self , inputs):
    return self.activation(keras.ops.conv(inputs, self.kernel) + self.bias)


In [80]:
myconv = MyConv2D(filters=32, kernel_size=3, activation="relu")

In [81]:
myconv.get_weights()

[]

Je krijgt een lege lijst?

De build is hier nog niet uitgevoerd dus get_weights is nog leeg

Als je de laag toepast op een tensor (bijvoorbeeld X), dan kan die build wel uitgevoerd worden

In [82]:
myconv(X).shape

Calling build


TensorShape([1, 126, 126, 32])

TensorShape([1, 126, 126, 32])

Hoe kom je aan die shape?

* 1: X is één item
* 126: kernel size 3 met padding valid -> we verliezen aan elke kant één pixel
* 32: aantal filters dat we gegeven hebben

In [83]:
myconv(X).shape

TensorShape([1, 126, 126, 32])

Calling build print niet meer te zien want build wordt maar één keer uitgevoerd!

In [84]:
myconv.get_weights()

[array([[[[-1.15430266e-01, -6.05370924e-02,  4.21942137e-02, ...,
           -4.80104089e-02, -1.00457974e-01,  9.30023286e-03],
          [-9.82313976e-02, -1.24817796e-01,  7.46175572e-02, ...,
            1.41660020e-01,  5.53729832e-02, -2.21327916e-02],
          [-8.75273719e-02, -1.60908159e-02,  2.58096531e-02, ...,
            7.22476989e-02,  2.12768130e-02, -5.83182089e-02],
          ...,
          [-9.67523956e-04,  6.45781606e-02, -7.71478983e-03, ...,
           -9.03610662e-02, -2.09444705e-02, -5.46768233e-02],
          [-7.90981948e-02, -4.81862612e-02, -7.60335922e-02, ...,
            5.30198924e-02, -3.08809411e-02, -2.54613478e-02],
          [-2.95090564e-02, -6.77770078e-02, -8.65569264e-02, ...,
           -3.22926417e-02, -5.62628964e-04,  5.14337532e-02]],
 
         [[ 2.07192376e-02,  6.82664663e-02,  6.17827429e-03, ...,
            5.05179055e-02,  6.46585971e-02, -6.55956939e-02],
          [ 6.68438822e-02, -1.26632024e-02, -4.09707613e-02, ...,
     

Nu zijn er wel gewichten te zien omdat build uitgevoerd is!

You can now use this layer as any other layer in Keras.

**TODO**: Check that it yields identical results as a Conv2D layer (after copying the weights from one layer to the
other).